<a href="https://www.kaggle.com/code/joshuaokolo/nlp-and-topic-modeling-for-infrastructure-needs?scriptVersionId=104056109" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Data Cleaning

In [ ]:
from langdetect import detect
import re
emoji_pattern = re.compile("["
   u"\U0001F600-\U0001F64F" # emoticons
   u"\U0001F300-\U0001F5FF" # symbols & pictographs
   u"\U0001F680-\U0001F6FF" # transport & map symbols
   u"\U0001F1E0-\U0001F1FF" # flags (iOS)
   u"\U00002702-\U000027B0"
   u"\U000024C2-\U0001F251"
   u"\U00002500-\U00002BEF" # chinese char
   u"\U0001f921-\U0001f937"
   u"\U00010000-\U0010ffff"
   u"\u2640-\u2642"
   u"\u2600-\u2B55"
   u"\u200d"
   u"\u23cf"
   u"\u23e9"
   u"\u231a"
   u"\ufe0f" # dingbats
   u"\u3030"
   "]+", flags=re.UNICODE)
email_pattern = re.compile("\S+@\S+\.\S{2,3}")
link_pattern = re.compile("https?\S+")

In [ ]:
def clean_data(tweet):
    try:
        lang = detect(tweet)
    if lang == 'en':
        tweet_rep = emoji_pattern.sub(r'', tweet)
        tweet_rep = email_pattern.sub(r'', tweet_rep)
        tweet_rep = link_pattern.sub(r'', tweet_rep)
        tweet_rep = tweet_rep.replace("’", "‘")
        tweet_rep = tweet_rep.replace("&amp;", "&")
        tweet_rep = tweet_rep.replace("#", '')
        tweet_rep = tweet_rep.strip()
        return tweet_rep
    else:
        return ""
    except:
        return ""

## Topic Modeling

In [ ]:
import nltk; nltk.download('stopwords')
import re
import numpy as npimport pandas as pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Data Preprocessing

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuationsdata_words = list(sent_to_words(data))
print(data_words[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)# See trigram example

print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=[‘NOUN’, ‘ADJ’, ‘VERB’, ‘ADV’]):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(“ “.join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
# Perform lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[‘NOUN’, ‘ADJ’, ‘VERB’, ‘ADV’])

print(data_lemmatized[:1])In the lemmatization step, we only keep words that belong to one of the following parts of speech: noun, adjective, verb, adverb. Hence, you can expect to get the output that look as follows:

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Next, we apply the doc2bow function to convert the texts into the bag-of-words (BoW) format, 
# Which is a list of (token_id, token_count) tuples.

# Create a corpus from the lemmatized text we want to analyse
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

## LDA Model

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
            id2word=id2word,
            num_topics=5,
            random_state=100,
            update_every=1,
            chunksize=50,
            passes=20,
            alpha=’auto’,
            per_word_topics=True)

In [ ]:
print(lda_model.print_topics())

## LDAMallet

In [ ]:
!curl http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip — output mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [ ]:
mallet_path = ‘/content/mallet-2.0.8/bin/mallet’ # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)

# To show the topics
ldamallet.show_topics()

## Optimize number of topics with coherence scores

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
  """
  Compute c_v coherence for various number of topics
  Parameters:
  — — — — —
  dictionary : Gensim dictionary
  corpus : Gensim corpus
  texts : List of input texts
  limit : Max num of topics
  Returns:
  — — — -
  model_list : List of LDA topic models
  coherence_values : Coherence values corresponding to the LDA model with respective number of topics
  """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=’c_v’)
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=80, step=6)

In [ ]:
# Show graph
limit=80; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel(“Num Topics”)
plt.ylabel(“Coherence score”)
plt.legend((“coherence_values”), loc=’best’)
plt.show()

## Visualizaiton

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
  # Init output
  sent_topics_df = pd.DataFrame()
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
    # Get the Dominant topic, Perc Contribution and Keywords for each document
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0: # => dominant topic
            wp = ldamodel.show_topic(topic_num)
            topic_keywords = “, “.join([word for word, prop in wp])
            sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
        else:
            break
    sent_topics_df.columns = [‘Dominant_Topic’, ‘Perc_Contribution’, ‘Topic_Keywords’]
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = [‘Document_No’, ‘Dominant_Topic’, ‘Topic_Perc_Contrib’, ‘Keywords’, ‘Text’]
# Show
df_dominant_topic.head(20)

In [ ]:
df_dominant_topic[‘Dominant_Topic’]==0.0 (use 1.0 for topic 2, 2.0 for topic 3, etc.).
topic1data = df_dominant_topic[df_dominant_topic[‘Dominant_Topic’]==0.0]
keyw_topic1 = []

    for each in df_dominant_topic['Keywords']:
        for l in each.strip().split(“,”):
            keyw_topic1.append(l.strip())

keyw_topic1 = list(set(keyw_topic1))
from wordcloud import WordCloud, STOPWORDS

df_dominant_topic[‘Dominant_Topic’].unique()
topic1data = df_dominant_topic[df_dominant_topic[‘Dominant_Topic’]==0.0]
comment_words = ''
stopwords = set(STOPWORDS)

# iterate through the csv file
for val in topic1data.Text:
  # typecast each val to string
  val = str(val)
  # split the value
  tokens = val.split()
  # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
    comment_words += “ “.join(tokens)+” “

wordcloud = WordCloud(width = 800, height = 800,
                      background_color =’white’,
                      stopwords = stopwords,
                      min_font_size = 10).generate(comment_words)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis(“off”)
plt.tight_layout(pad = 0)
plt.show()

References:

https://radimrehurek.com/gensim/models/ldamodel.html

https://github.com/twintproject/twint

https://ngrok.com

https://pypi.org/project/colabcode/

https://omdena.com/blog/infrastructural-needs/

https://pypi.org/project/langdetect/